# Análise Licitacões Públicas CGU

In [4]:
import os
from dotenv import load_dotenv
import pandas as pd
import basedosdados as bd

load_dotenv()
billing_id = os.getenv("BILLING_ID")

O número de licitações públicas segue um padrão mensal ou varia significativamente ao longo do tempo ?

In [5]:
query = """
  SELECT
  FORMAT_DATE('%Y-%m', data_abertura) AS ano_mes,
  COUNT(DISTINCT id_licitacao) AS total_licitacoes
FROM basedosdados.br_cgu_licitacao_contrato.licitacao
WHERE data_abertura IS NOT NULL
GROUP BY ano_mes
ORDER BY ano_mes DESC;

"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|


,ano_mes,total_licitacoes
0,2023-10,283
1,2023-09,385
2,2023-08,408
3,2023-07,359
4,2023-06,333
...,...,...
138,2012-03,1
139,2011-12,2
140,2011-11,1
141,2011-09,1


Quais órgãos públicos mais licitam e movimentam recursos em contratações?"



In [6]:
query = """
 SELECT
  l.nome_orgao,
  COUNT(DISTINCT l.id_licitacao) AS total_licitacoes,
  ROUND(SUM(li.valor_item), 2) AS valor_total_licitado
FROM basedosdados.br_cgu_licitacao_contrato.licitacao AS l
JOIN basedosdados.br_cgu_licitacao_contrato.licitacao_item AS li
  ON l.id_licitacao = li.id_licitacao
GROUP BY l.nome_orgao
ORDER BY total_licitacoes DESC, valor_total_licitado DESC
LIMIT 10;

"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading:  18%|█▊        |


KeyboardInterrupt: 

Qual é o nível médio de concorrência nas licitações?

In [ ]:
query = """
 SELECT
  ROUND(AVG(numero_concorrentes), 2) AS media_participantes_por_licitacao
FROM (
  SELECT
    id_licitacao,
    COUNT(DISTINCT cpf_cnpj_participante) AS numero_concorrentes
  FROM basedosdados.br_cgu_licitacao_contrato.licitacao_participante
  GROUP BY id_licitacao
);

"""

bd.read_sql(query = query, billing_project_id = billing_id)


Downloading: 100%|██████████|


,media_participantes_por_licitacao
0,57.68


Quais modalidades de licitação concentram os itens mais caros?"

In [ ]:
query = """
SELECT
  modalidade,
  ROUND(AVG(valor_item), 2) AS valor_medio_item
FROM basedosdados.br_cgu_licitacao_contrato.licitacao_item
GROUP BY modalidade
ORDER BY valor_medio_item DESC;

"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|


,modalidade,valor_medio_item
0,Pregão - Registro de Preço,10700062.32
1,Concorrência,4692230.20
2,Concorrência Internacional,547987.29
3,Tomada de Preços,338047.21
4,Inexigibilidade de Licitação,333987.12
5,Concorrência Internacional - Registro de Preço,180705.09
6,Pregão,116111.02
7,Dispensa de Licitação,54651.31
8,Concorrência - Registro de Preço,47432.55
9,Convite,31116.06


Quais participantes têm mais vitórias em licitações dentro de cada órgão?

In [ ]:
query = """
SELECT
  lp.nome_participante,
  l.nome_orgao,
  SUM(CASE WHEN lp.vencedor = TRUE THEN 1 ELSE 0 END) AS total_vitorias
FROM basedosdados.br_cgu_licitacao_contrato.licitacao_participante AS lp
JOIN basedosdados.br_cgu_licitacao_contrato.licitacao AS l
  ON lp.id_licitacao = l.id_licitacao
WHERE lp.vencedor = TRUE
GROUP BY lp.nome_participante, l.nome_orgao
ORDER BY total_vitorias DESC;

"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading:  18%|█▊        |

GenericGBQException: Reason: Timeout of 600.0s exceeded, last exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

 Valor Total e Médio Contratado por Mês



In [7]:
query = """
SELECT
  FORMAT_DATE('%Y-%m', data_assinatura_contrato) AS ano_mes_contrato,
  ROUND(SUM(valor_final_compra), 2) AS valor_contratado_mensal_total,
  ROUND(AVG(valor_final_compra), 2) AS valor_medio_contratado_mensal
FROM basedosdados.br_cgu_licitacao_contrato.contrato_compra
WHERE data_assinatura_contrato IS NOT NULL
  AND valor_final_compra > 0
GROUP BY ano_mes_contrato
ORDER BY ano_mes_contrato DESC;
"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|


,ano_mes_contrato,valor_contratado_mensal_total,valor_medio_contratado_mensal
0,2103-07,2.891265e+05,2.891265e+05
1,2024-11,1.306596e+04,1.306596e+04
2,2024-07,1.810014e+09,3.036936e+06
3,2024-06,7.334726e+09,3.502735e+06
4,2024-05,5.367431e+09,2.187217e+06
...,...,...,...
212,2002-09,1.040000e+07,1.040000e+07
213,2000-04,4.918500e+05,4.918500e+05
214,1997-04,1.418669e+05,1.418669e+05
215,1996-08,6.000000e+03,6.000000e+03


Itens Mais Demandados e seus Valores

In [8]:
query = """
SELECT
  descricao_item,
  COUNT(DISTINCT id_licitacao) AS total_licitacoes_item,
  ROUND(SUM(valor_item), 2) AS valor_total_item_licitado,
  ROUND(AVG(valor_item), 2) AS valor_medio_item
FROM basedosdados.br_cgu_licitacao_contrato.licitacao_item
GROUP BY descricao_item
ORDER BY total_licitacoes_item DESC, valor_total_item_licitado DESC;
"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|


,descricao_item,total_licitacoes_item,valor_total_item_licitado,valor_medio_item
0,PAGAMENTO INSCRICAO EVENTOS,12355,1.922021e+08,2990.96
1,CURSO APERFEICOAMENTO / ESPECIALIZACAO PROFISS...,4628,4.494342e+08,14132.26
2,LOCACAO DE IMOVEL,4511,3.494552e+09,281025.49
3,PRESTACAO DE SERVICOS DE APOIO ADMINISTRATIVO,4494,2.392312e+10,824906.56
4,ASSISTENCIA MEDICA - HOSPITALAR / DOMICILIAR C...,4352,2.365286e+10,876096.58
...,...,...,...,...
16153,AÇAIMO,1,2.600000e+00,2.60
16154,CIRURGIA APARELHO DIGESTIVO E ORGAOS ANEXOS - ...,1,1.000000e-02,0.01
16155,CIRURGIA APARELHO DIGESTIVO E ORGAOS ANEXOS - ...,1,1.000000e-02,0.01
16156,CONSULTA MEDICA - HEPATOLOGIA,1,1.000000e-02,0.01


Sazonalidade nas Assinaturas de Contratos

In [9]:
query = """
SELECT
  FORMAT_DATE('%Y-%m', data_assinatura_contrato) AS ano_mes_assinatura,
  COUNT(DISTINCT id_contrato) AS total_contratos_assinados
FROM basedosdados.br_cgu_licitacao_contrato.contrato_compra
WHERE data_assinatura_contrato IS NOT NULL
GROUP BY ano_mes_assinatura
ORDER BY ano_mes_assinatura DESC;
"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|


,ano_mes_assinatura,total_contratos_assinados
0,2103-07,1
1,2024-11,1
2,2024-07,209
3,2024-06,462
4,2024-05,491
...,...,...
215,1999-12,1
216,1997-08,1
217,1997-04,1
218,1996-08,1


Situação das Licitações por Órgão

In [10]:
query = """
SELECT
  nome_orgao,
  COUNT(DISTINCT id_licitacao) AS total_registros,
  SUM(CASE WHEN situacao_licitacao = "Publicado" THEN 1 ELSE 0 END) AS total_publicado,
  SUM(CASE WHEN situacao_licitacao = "Inválido" THEN 1 ELSE 0 END) AS total_invalido,
  SUM(CASE WHEN situacao_licitacao = "Revogação" THEN 1 ELSE 0 END) AS total_revogacao,
  SUM(CASE WHEN situacao_licitacao = "Anulação" THEN 1 ELSE 0 END) AS total_anulacao
FROM basedosdados.br_cgu_licitacao_contrato.licitacao
WHERE situacao_licitacao IN ('Anulação', 'Revogação','Publicado', 'Inválido', 'Encerrado')
GROUP BY nome_orgao
ORDER BY total_registros DESC;
"""

bd.read_sql(query = query, billing_project_id = billing_id)

Downloading: 100%|██████████|


,nome_orgao,total_registros,total_publicado,total_invalido,total_revogacao,total_anulacao
0,Universidade Federal do Rio Grande do Sul,24815,2884,1,3,314
1,Fundação Universidade de Brasília,18143,3675,0,17,12
2,Universidade Federal do Pará,16223,2192,1,17,22
3,Comando da Marinha,14627,16660,22,166,370
4,Comissão Nacional de Energia Nuclear,12282,1714,3,57,97
...,...,...,...,...,...,...
270,Fundo Nacional de Habitação de Interesse So,1,1,0,0,0
271,CONSELHO REG. DE ARQUITETURA E URBANISMO-ES,1,0,0,0,0
272,UNIVERSIDADE FEDERAL DE ALAGOAS,1,0,0,0,0
273,Fundo de Defesa de Direitos Difusos,1,1,0,0,0


Volume de Licitações por Órgão e UF

In [ ]:
query = """
WITH licitacao_uf AS (
  SELECT
    l.nome_orgao,
    l.sigla_uf,
    COUNT(DISTINCT l.id_licitacao) AS total_licitacao
  FROM basedosdados.br_cgu_licitacao_contrato.licitacao AS l
  JOIN basedosdados.br_cgu_licitacao_contrato.licitacao_participante AS lp
    ON l.id_licitacao = lp.id_licitacao
  GROUP BY l.nome_orgao, l.sigla_uf
)
SELECT
  nome_orgao,
  sigla_uf,
  total_licitacao
FROM licitacao_uf
ORDER BY total_licitacao DESC;

"""

bd.read_sql(query = query, billing_project_id = billing_id)

 Itens Mais Comuns em Licitações de Alto Valor

In [ ]:
query = """
WITH LicitacaoPorFaixa AS (
    SELECT
        id_licitacao,
        SUM(valor_item) AS valor_licitacao,
        CASE
            WHEN SUM(valor_item) >= 1000000 THEN 'Alto Valor (>1M)'
            WHEN SUM(valor_item) >= 500000 AND SUM(valor_item) < 1000000 THEN 'Medio-Alto Valor (500k-1M)'
            ELSE 'Outros Valores'
        END AS faixa_valor_licitacao
    FROM basedosdados.br_cgu_licitacao_contrato.licitacao_item
    GROUP BY id_licitacao
    HAVING SUM(valor_item) >= 500000
),
ItemEmLicitacaoAltoValor AS (
    SELECT
        li.descricao_item,
        lf.faixa_valor_licitacao,
        COUNT(DISTINCT li.id_licitacao) AS total_ocorrencias_item
    FROM basedosdados.br_cgu_licitacao_contrato.licitacao_item AS li
    JOIN LicitacaoPorFaixa AS lf
        ON li.id_licitacao = lf.id_licitacao
    GROUP BY
        li.descricao_item,
        lf.faixa_valor_licitacao
),
RankingItensPorFaixa AS (
    SELECT
        descricao_item,
        faixa_valor_licitacao,
        total_ocorrencias_item,
        ROW_NUMBER() OVER (PARTITION BY faixa_valor_licitacao ORDER BY total_ocorrencias_item DESC) AS rank_item_na_faixa
    FROM ItemEmLicitacaoAltoValor
)
SELECT
    descricao_item,
    faixa_valor_licitacao,
    total_ocorrencias_item
FROM RankingItensPorFaixa
WHERE rank_item_na_faixa <= 10
ORDER BY
    faixa_valor_licitacao DESC,
    total_ocorrencias_item DESC
LIMIT 10
"""

bd.read_sql(query = query, billing_project_id = billing_id)

 Classificação dos Itens por Setor e UF

In [ ]:
query = """
WITH LicitacaoPorFaixa AS (
    SELECT
        id_licitacao,
        SUM(valor_item) AS valor_licitacao,
        CASE
            WHEN SUM(valor_item) >= 1000000 THEN 'Alto Valor (>1M)'
            WHEN SUM(valor_item) >= 500000 AND SUM(valor_item) < 1000000 THEN 'Medio-Alto Valor (500k-1M)'
            ELSE 'Outros Valores'
        END AS faixa_valor_licitacao
    FROM basedosdados.br_cgu_licitacao_contrato.licitacao_item
    GROUP BY id_licitacao
    HAVING SUM(valor_item) >= 500000
),
ItemEmLicitacaoAltoValor AS (
    SELECT
        li.descricao_item,
        lf.faixa_valor_licitacao,
        COUNT(DISTINCT li.id_licitacao) AS total_ocorrencias_item
    FROM basedosdados.br_cgu_licitacao_contrato.licitacao_item AS li
    JOIN LicitacaoPorFaixa AS lf
        ON li.id_licitacao = lf.id_licitacao
    GROUP BY
        li.descricao_item,
        lf.faixa_valor_licitacao
),
RankingItensPorFaixa AS (
    SELECT
        descricao_item,
        faixa_valor_licitacao,
        total_ocorrencias_item,
        ROW_NUMBER() OVER (PARTITION BY faixa_valor_licitacao ORDER BY total_ocorrencias_item DESC) AS rank_item_na_faixa
    FROM ItemEmLicitacaoAltoValor
)
SELECT
    descricao_item,
    faixa_valor_licitacao,
    total_ocorrencias_item
FROM RankingItensPorFaixa
WHERE rank_item_na_faixa <= 10
ORDER BY
    faixa_valor_licitacao DESC,
    total_ocorrencias_item DESC
LIMIT 10
"""

bd.read_sql(query = query, billing_project_id = billing_id)